In [ ]:
!pip install langchain # langchain 활용
!pip install openai==0.27.8 # chatgpt api 활용
!pip install google-search-results # Google Search 결과를 가져오기 위한 라이브러리
!pip install wikipedia # Wikipedia API를 사용하기 위한 라이브러리
!pip install faiss-cpu # FAISS는 Facebook에서 만든 오픈소스 벡터 데이터베이스 라이브러리 (MIT license)
!pip install sentence_transformers # HuggingFace Embedding 사용 위해서 필요
!pip install tiktoken # 텍스트를 요약할 때 필요한 라이브러리
!pip install PyPDF2 # PDF 파일을 읽어오기 위한 라이브러리

In [11]:
#@title API 키 설정
import os
#@markdown https://platform.openai.com/account/api-keys
#chatGPT를 활용할 때 사용하였던 api 키를 입력해 주세요
OPENAI_API_KEY = "" #@param {type:"string"}

#@markdown https://huggingface.co/settings/tokens
#@markdown HuggingFace에서 모델 다운로드나 클라우드 모델 사용하기 위해서 필요 (무료)
# huggingFace api 키를 입력해 주세요
HUGGINGFACEHUB_API_TOKEN = "" #@param {type:"string"}

#@markdown https://serpapi.com/manage-api-key
#@markdown 구글 검색하기 위해서 필요 (월 100회 무료)
#serpapi에서 발급받은 api 키를 입력해 주세요
SERPAPI_API_KEY = "" #@param {type:"string"}

#API KEY를 숨기는 코드
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

In [19]:
#!pip install langchain langchain_community
from langchain.llms import OpenAI  #옛날버전

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

llm("90년대 음악을 추천해줘")

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:1072: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


'1. Nirvana - "Smells Like Teen Spirit"\n2. Oasis - "Wonderwall"\n3. Spice Girls - "Wannabe"\n4. Backstreet Boys - "I Want It That Way"\n5. TLC - "Waterfalls"\n6. Radiohead - "Creep"\n7. No Doubt - "Don\'t Speak"\n8. Pearl Jam - "Jeremy"\n9. R.E.M. - "Losing My Religion"\n10. Mariah Carey - "Fantasy"'

In [6]:
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,  # AI가 보내는 메시지를 나타내는 클래스
    SystemMessage,  # AI의 동작이나 컨텍스트를 설정하는 데 사용되는 시스템 메시지 클래스
    HumanMessage  # 사람이 보내는 메시지를 나타내는 클래스
)

# GPT-3.5-turbo 모델을 사용하는 채팅 모델을 초기화합니다. temperature로 생성된 텍스트의 다양성을 조절
chat = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.9)
# 시스템 메시지를 생성하여 AI의 역할을 "당신은 영화 추천을 해주는 전문 AI입니다"로 설정
sys = SystemMessage(content="당신은 영화 추천을 해주는 전문 AI입니다")
# 사람이 보내는 메시지를 생성하여 "2000년대 액션영화 5개를 추천해줘"라는 요청을 보냄
msg = HumanMessage(content="2000년대 액션영화 5개를 추천해줘")

# 시스템 메시지와 사람 메시지를 기반으로 AI에게 요청을 보내고, AI의 응답을 aimsg에 저장
aimsg = chat([sys,msg])
print(aimsg.content) #AI응답내용 출력



ModuleNotFoundError: No module named 'langchain_community'

In [12]:
from langchain.prompts import PromptTemplate

# PromptTemplate 객체 생성
prompt = PromptTemplate(
    input_variables=["상품"], # 템플릿에서 사용할 변수 지정
    template="{상품}유행어를 기반으로 재밌는 메뉴 이름 추천해줘"  # 해당 변수들을 사용하여 최종 메시지를 생성할 형식을 지정
)

# format 메서드를 사용하여 '상품' 변수에 값을 할당하고 최종 메시지를 생성
prompt.format(상품="AI 치킨 추천 서스")

'AI 치킨 추천 서스유행어를 기반으로 재밌는 메뉴 이름 추천해줘'

In [27]:
from langchain.chains import LLMChain

# LLMChain 객체를 생성합니다. 위에서 정의한 chat 모델과 prompt 템플릿을 사용
chain = LLMChain(llm=chat, prompt=prompt)

result = chain.run("상품=AI 치킨 추천")

print(result)

1. AI 치킨 맛집
2. 미래의 맛, AI 치킨
3. 인공지능이 뽑은 최고의 치킨
4. AI 치킨의 비밀
5. 미래의 맛, AI 치킨킹
6. 인공지능의 선택, AI 치킨파이터
7. AI가 고른 최고의 치킨
8. AI 치킨의 세계
9. 미래를 맛보는 AI 치킨
10. AI 치킨의 맛있는 비밀


In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

chat = ChatOpenAI(temperature=0)

# 시스템 메시지 템플릿 정의. 이 템플릿은 입력 언어와 출력 언어를 받아 번역하는 역할을 합니다.
template = "You are a helpful assistant that translate {input_language} to {output_language}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# 사람 메시지 템플릿 정의. 이 템플릿은 번역할 텍스트를 포함합니다.
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 시스템 메시지와 사람 메시지를 포함하는 채팅 프롬프트 템플릿 생성
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
# ChatOpenAI 모델과 채팅 프롬프트 템플릿을 사용하여 LLMChain 객체를 생성
chatchain = LLMChain(llm=chat, prompt=chat_prompt)
# LLMChain의 run 메서드를 사용하여 번역 작업을 수행. 여기서는 영어 텍스트를 한국어로 번역합니다.
chatchain.run(input_language="English", output_language="Korean", text="I love Programming")


'저는 프로그래밍을 좋아해요.'

In [53]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

chat = ChatOpenAI(temperature=0)

tools = load_tools(["serpapi","llm-math"],llm=chat) #특정 일을 하는 존재 예)구글 검색, 디비 조회, 수학 게산
agent = initialize_agent(tools, llm=chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) #어떤 도구를 어떤 순서로 실행할지 결정하는 존재 (LLM이용)

# response =agent.run("Google CEO와 Tesla CEO의 나이 차이는 어떻게 되나요?")
# print(response)

response = agent.run("Google CEO와 테슬라 CEO 둘의 나이 차이는?")
print(response)



> Entering new AgentExecutor chain...
I should use a search engine to find the ages of the Google CEO and Tesla CEO.
Action: Search
Action Input: "Google CEO age" 
Observation: Sundar Pichai (born June 10, 1972, Madras [now Chennai], Tamil Nadu, India) is an Indian-born American executive who was CEO of both Google, Inc. (2015– ), and its holding company, Alphabet Inc. (2019– ).
Thought:I need to find the age of the Tesla CEO now.
Action: Search
Action Input: "Tesla CEO age"
Observation: ['Elon Reeve Musk. (1971-06-28) June 28, 1971 (age 53). Pretoria, Transvaal, South Africa · South Africa; Canada; United States · University of Pennsylvania (BA, BS).', 'Forbes Lists ; Age: 53 ; Source of Wealth: Tesla, SpaceX, Self Made ; Self-Made Score: 8 ; Philanthropy Score: 1 ; Residence: Austin, Texas.', '(1971-06-28) June 28, 1971 (age 53). Pretoria, South Africa. Citizenship ... Musk is the current CEO & Chief Product Architect of Tesla, Inc., a company ...', "At age 12 he created a video ga

In [18]:
# 1.웹 페이지의 내용을 가져옴
# 2. 가져온 문서를 특정 크기로 분할
# 3. 분할된 문서를 바탕으로 요약하는 chain 실행
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

# 1.웹페이지 내용 가져오기 위해 WebBaseLoader 초기화
loader = WebBaseLoader(web_path='https://ko.wikipedia.org/wiki/NewJeans')
documents = loader.load()

# 2. 분할하기 위해 CharacterTextSplitter 초기화
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100, length_function=len) # chunk_size는 각 청크(조각)의 길이를 지정하고, chunk_overlap은 청크 간의 중첩 부분을 지정
docs = text_splitter.split_documents(documents)

#3.ChatOpenAI 모델 초기화

template = '''
다음의 내용을 한글로 요약해줘
{text}
'''
combine_template = '''
{text}
내용에 대한 요약은 markdown 형식으로 해줘
'''

prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
chain = load_summarize_chain(chat, chain_type='map_reduce',verbose=False, map_prompt=prompt, combine_prompt=combine_prompt)  # verbose=True로 하면 중간과정을 볼 수 있음
# map_reduce - 나눠서 요약을 진행하고, 나눠서 한 요약 전체를 한번 더 요약함
# stuff - 한번에 다 넣고 요약 / 메모리 문제로 오류 날 확률 높음
# map_rerank - 순위를 매겨서 중요한 거 위주로 요약

chain.run(docs[:5])

"**NewJeans**\n\n- **활동**: 대한민국의 음악 그룹으로, 2011년에 데뷔 전 활동을 시작으로 현재까지 활발히 활동 중이다. 다양한 음반을 발표하고 텔레비전 방송 프로그램에 출연하며 수상도 다수 했다.\n  \n- **앨범**: 최근에는 'Get Up'이라는 앨범을 발표하며 화제를 모으고 있다.\n  \n- **멤버**: 5인조 걸그룹으로, 하이브 소속사 어도어에서 활동하며, SM 엔터테인먼트 비주얼 디렉터 출신 민희진이 프로듀서로 활동 중이다.\n  \n- **그룹 이름**: '뉴진스(NewJeans)'는 청바지인 '진스(jeans)'의 이미지와 새로운 세대를 대표하는 'new genes'의 뜻을 함축한 그룹 이름이다."

In [ ]:
#4-9 PDF로 학습하는 ChatGPT
!pip -q install langchain openai==0.27.8 tiktoken PyPDF2 faiss-cpu cohere

#허생전 PDF
# https://s3.ap-northeast-2.amazonaws.com/materials.spartacodingclub.kr/AI_jonghap/%EC%B1%84%EB%A7%8C%EC%8B%9D-%ED%97%88%EC%83%9D%EC%A0%84.pdf



In [ ]:
!pip install -U langchain-community


from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

from google.colab import drive
drive.mount('/content/gdrive')
doc_reader = PdfReader('/content/gdrive/MyDrive/인공지능/채만식-허생전.pdf')


# 파일 읽어와서 raw_text 변수에 넣기
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

# print(raw_text[:100])
# print(len(raw_text))

# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 500
)
texts = text_splitter.split_text(raw_text)
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)

query="이 소설의 주인공이 누구야?"
docs = docsearch.similarity_search(query) # 소설의 주인공과 관련있는 chunk를 가지고 옴

chain = load_qa_chain(OpenAI(),chain_type="map_reduce")

response = chain.run(input_documents=docs, question=query)
print(response)

In [22]:
new_query = "허생은 왜 변진사를 찾아갔어?"
new_docs=docsearch.similarity_search(new_query)
new_response=chain.run(input_documents=new_docs, question=new_query)
print(new_response)

 허생은 가족을 거느리고 충청도 강경장터로 가기 위해 변진사를 찾아갔다.
